In [1]:
from copy import deepcopy


import numpy as np
import pytest
from gl0learn import fit, synthetic, Penalty
from gl0learn.metrics import nonzeros, pseudo_likelihood_loss
from gl0learn.utils import triu_nnz_indicies
from hypothesis import given, settings, assume, HealthCheck, note
from hypothesis.strategies import just, booleans, floats, integers, random_module
MAX_OVERLAPS = 6


import sys
sys.path.append("tests/helper")

from utils import (
    sample_from_cov,
    overlap_covariance_matrix,
    is_scipy_installed,
    is_mosek_installed,
    make_bisect_func,
    random_penalty,
    random_penalty_values,
    top_n_triu_indicies_by_abs_value,
)


def test_cd_limited_active_set(p, seed):
    theta_truth = overlap_covariance_matrix(p=p, seed=seed, decay=0.8)
    x = sample_from_cov(theta_truth, n=1000)
    _, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)
    results = fit(
        x,
        l0=0,
        scale_x=True,
        max_active_set_ratio=1,
        active_set=np.inf,
        super_active_set=0.0,
    )

    theta_truth_copy = np.copy(theta_truth)
    np.fill_diagonal(theta_truth_copy, 0)
    i, j = np.unravel_index(np.argmax(theta_truth_copy), theta_truth.shape)

    assert results.theta[i, j] > np.mean(theta_truth_copy)


@pytest.mark.skipif(not is_scipy_installed(), reason="`scipy` is not installed.")
@pytest.mark.parametrize("nnz", range(1, 10))
@pytest.mark.parametrize("algorithm", ["CD", "CDPSI"])
@given(
    p=integers(3, 10),
    module=random_module(),
    lXs=random_penalty_values(
        penalty_strategies=random_penalty(l0=just(False), l1=booleans(), l2=booleans()),
        values_strategies={"l1": floats(0.01, 10), "l2": floats(0.01, 10)},
    ),
)
@settings(max_examples=100, deadline=None)
def test_cd_example_2(p, module, nnz, algorithm, lXs):
    theta_truth = overlap_covariance_matrix(p=p, seed=module.seed, decay=0.8)
    x = sample_from_cov(n=30 * p**2, cov=theta_truth)

    _, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    fit_kwargs = dict(
        **lXs,
        scale_x=False,
        max_active_set_ratio=1.0,
        active_set=0.0,
        super_active_set=0.0,
        algorithm=algorithm
    )

    f = make_bisect_func(nnz, Y, **fit_kwargs)

    from scipy.optimize import bisect

    try:
        opt_l0 = bisect(f, a=0, b=10)
    except ValueError:
        assume(False)

    results = fit(Y, l0=opt_l0, **fit_kwargs)

    theta = results.theta

    assume(nonzeros(theta[np.tril_indices(p, k=-1)]).sum() == nnz)

    cd_indices = top_n_triu_indicies_by_abs_value(results.theta, nnz)
    indices = top_n_triu_indicies_by_abs_value(theta_truth, nnz)

    if any(theta_truth[cd_indices] == 0):
        # CD algorithm has selected zero items. This can be fine if we ask for more non-zeros than are in theta_truth!
        # Check if indicies is contained in cd_indices
        indices_set = set(zip(*indices))
        cd_indices_set = set(zip(*cd_indices))
        assert cd_indices_set.issuperset(indices_set)
        should_be_zero_indices_set = cd_indices_set - indices_set

        for (i, j) in should_be_zero_indices_set:
            assert theta_truth[i, j] == 0

    else:
        np.testing.assert_array_equal(cd_indices, indices)


@pytest.mark.parametrize("algorithm", ["CD", "CDPSI"])
@given(
    p=integers(3, 10),
    module=random_module(),
    overlaps=integers(1, MAX_OVERLAPS - 1),
    lXs=random_penalty_values(
        penalty_strategies=random_penalty(l0=just(True), l1=booleans(), l2=booleans()),
        values_strategies={
            "l0": floats(0.01, 10),
            "l1": floats(0.01, 10),
            "l2": floats(0.01, 10),
        },
    ),
)
@settings(suppress_health_check=[HealthCheck(2)], deadline=None)
def test_super_active_set(algorithm, p, module, overlaps, lXs):
    # TODO: Figure out this hypothesis bug. When lXs aren't deep copied, the tracebacks provided by hypothesis are wrong.
    lX2s = deepcopy(lXs)
    theta_truth = overlap_covariance_matrix(
        p=p,
        seed=module.seed,
        max_overlaps=overlaps,
        decay=1 - np.exp(overlaps - MAX_OVERLAPS),
    )
    x = sample_from_cov(n=30 * p**2, cov=theta_truth)

    _, _, _, _, Y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    test_result = fit(
        Y, **lXs, active_set=np.inf, super_active_set=0.0, max_active_set_ratio=1.0
    )

    assume(test_result.active_set_size[-1] > 0)

    # replace with triu_nnz_indicies
    # print(test_result.theta)
    possible_active_set = np.asarray(
        np.where(np.abs(np.triu(test_result.theta, k=1)) > 0)
    ).T

    active_set_size = possible_active_set.shape[0]
    # print(active_set_size)
    if possible_active_set.shape[0] > 1:
        num_selected = np.random.randint(1, active_set_size)
        idx = np.sort(
            np.random.choice(
                np.arange(active_set_size), size=num_selected, replace=False
            )
        )
    else:
        num_selected = 1
        idx = [0]

    # print(idx)
    # print(possible_active_set)

    initial_super_active_set = possible_active_set[idx, :]

    lX2s["l0"] = 0

    theta_init = np.diag(np.diag(test_result.theta))
    for row, col in initial_super_active_set:
        theta_init[row, col] = theta_init[col, row] = test_result.theta[row, col]

    results = fit(
        Y,
        **lX2s,
        theta_init=theta_init,
        active_set=initial_super_active_set,
        super_active_set=initial_super_active_set,
        max_active_set_ratio=1.0
    )

    cd_indices = top_n_triu_indicies_by_abs_value(results.theta, num_selected)

    np.testing.assert_array_equal(np.asarray(cd_indices).T, initial_super_active_set)


@pytest.mark.skipif(not is_mosek_installed(), reason="`mosek` is not installed.")
@given(
    p=integers(3, 10),
    n=floats(0, 1000),
    overlaps=integers(1, MAX_OVERLAPS - 1),
    module=random_module(),
    lXs=random_penalty_values(
        penalty_strategies=random_penalty(l0=just(True), l1=just(False), l2=just(True)),
        values_strategies={"l0": floats(0.1, 10), "l2": floats(0.1, 10)},
    ),
)
@settings(deadline=None, suppress_health_check=[HealthCheck(2)])
def test_cd_vs_mosek(n, p, module, overlaps, lXs):
    from gl0learn.opt import MIO_mosek
    from mosek.fusion import SolutionError

    num_samples = max(1, int(n * p**2))
    theta_truth = overlap_covariance_matrix(
        p=p,
        seed=module.seed,
        max_overlaps=overlaps,
        decay=1 - np.exp(overlaps - MAX_OVERLAPS),
    )

    assume(all(np.linalg.eigvalsh(theta_truth) > 0))
    x = sample_from_cov(n=num_samples, cov=theta_truth)

    _, _, _, _, y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    m = np.max(np.abs(theta_truth * (1 - np.eye(p))))
    int_tol = 1e-4

    try:
        MIO_results = MIO_mosek(y=y, m=m, **lXs, int_tol=int_tol, max_time=10)
    except SolutionError:
        assume(False)

    cd_results = fit(
        y,
        **lXs,
        scale_x=False,
        theta_init=None,
        active_set=0.0,
        max_iter=1000,
        super_active_set=0.0,
        max_active_set_ratio=1.0,
        tol=1e-12
    )

    MIO_active_set = triu_nnz_indicies(MIO_results.theta_hat)
    CD_active_set = triu_nnz_indicies(cd_results.theta)

    if MIO_active_set.shape != CD_active_set.shape:
        assume(False)
    else:
        assume((MIO_active_set == CD_active_set).all())

    penalty = Penalty(**lXs)
    MIO_loss = pseudo_likelihood_loss(
        y, np.array(MIO_results.theta_hat), penalty, active_set=MIO_active_set
    )
    cd_loss = pseudo_likelihood_loss(
        y, np.array(cd_results.theta), penalty, active_set=CD_active_set
    )

    assert cd_loss <= MIO_loss + 1e-6 * abs(MIO_loss)


@pytest.mark.skipif(not is_mosek_installed(), reason="`mosek` is not installed.")
@pytest.mark.parametrize("max_iter", [1, 1000])
@pytest.mark.parametrize("algorithm", ["CD", "CDPSI"])
@given(
    n=integers(3, 1000),
    p=integers(3, 10),
    overlaps=integers(1, MAX_OVERLAPS - 1),
    module=random_module(),
    lXs=random_penalty_values(
        penalty_strategies=random_penalty(l0=just(True), l1=just(False), l2=just(True)),
        values_strategies={"l0": floats(0.1, 10), "l2": floats(0.1, 10)},
    ),
)
@settings(max_examples=250, deadline=None)
def test_cd_keeps_mio_results(max_iter, algorithm, n, p, module, overlaps, lXs):
    from gl0learn.opt import MIO_mosek

    theta_truth = overlap_covariance_matrix(
        p=p,
        seed=module.seed,
        max_overlaps=overlaps,
        decay=1 - np.exp(overlaps - MAX_OVERLAPS),
    )

    assume(all(np.linalg.eigvalsh(theta_truth) > 0))
    x = sample_from_cov(n=n, cov=theta_truth)

    _, _, _, _, y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    m = np.max(np.abs(theta_truth * (1 - np.eye(p))))
    int_tol = 1e-4

    MIO_results = MIO_mosek(y=y, m=m, **lXs, int_tol=int_tol)
    cd_results = fit(
        y,
        **lXs,
        scale_x=False,
        theta_init=MIO_results.theta_hat,
        max_iter=max_iter,
        algorithm=algorithm,
        active_set=0.0,
        super_active_set=0.0,
        max_active_set_ratio=1.0
    )

    try:
        np.testing.assert_array_equal(MIO_results.theta_hat, cd_results.theta)
    except AssertionError:
        penalty = Penalty(**lXs)
        active_set = np.asarray(np.triu_indices(p, k=1), order="C", dtype=np.uint64).T
        MIO_loss = pseudo_likelihood_loss(
            y, np.array(MIO_results.theta_hat), penalty, active_set=active_set
        )
        cd_loss = pseudo_likelihood_loss(
            y, np.array(cd_results.theta), penalty, active_set=active_set
        )

        assert cd_loss <= MIO_loss


@pytest.mark.skipif(not is_mosek_installed(), reason="`mosek` is not installed.")
@pytest.mark.parametrize("algorithm", ["CD", "CDPSI"])
@given(
    n=integers(3, 1000),
    p=integers(3, 10),
    overlaps=integers(1, MAX_OVERLAPS - 1),
    module=random_module(),
    lXs=random_penalty_values(
        penalty_strategies=random_penalty(l0=just(True), l1=just(False), l2=just(True)),
        values_strategies={"l0": floats(0.1, 10), "l2": floats(0.1, 10)},
    ),
)
@settings(max_examples=250, deadline=None)
def test_cd_learns_mio_results_from_support(algorithm, n, p, module, overlaps, lXs):
    # note({"n": n, "p": p, "module":module, overlaps: "overlaps", "lXs": lXs})
    from gl0learn.opt import MIO_mosek

    theta_truth = overlap_covariance_matrix(
        p=p,
        seed=module.seed,
        max_overlaps=overlaps,
        decay=1 - np.exp(overlaps - MAX_OVERLAPS),
    )

    assume(all(np.linalg.eigvalsh(theta_truth) > 0))
    x = sample_from_cov(n=n, cov=theta_truth)

    _, _, _, _, y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    m = np.max(np.abs(theta_truth * (1 - np.eye(p))))
    int_tol = 1e-4

    MIO_results = MIO_mosek(y=y, m=m, **lXs, int_tol=int_tol)
    active_set = triu_nnz_indicies(MIO_results.theta_hat)
    cd_results = fit(
        y,
        **lXs,
        scale_x=False,
        theta_init=None,
        max_iter=1000,
        algorithm=algorithm,
        active_set=active_set,
        super_active_set=active_set,
        max_active_set_ratio=1.0
    )

    penalty = Penalty(**lXs)
    MIO_loss = pseudo_likelihood_loss(
        y, np.array(MIO_results.theta_hat), penalty, active_set=active_set
    )
    cd_loss = pseudo_likelihood_loss(
        y, np.array(cd_results.theta), penalty, active_set=active_set
    )

    assert cd_loss <= MIO_loss + int_tol * abs(MIO_loss)


In [ ]:
test_cd_limited_active_set(3, 0)

Is this happening as we enter _fit?
0 <class 'numpy.ndarray'> [[ 2.07984015e-02 -1.71128848e-02 -2.70792682e-02 ...  2.32982245e-02
  -1.91273908e-02  3.57984356e-02]
 [ 1.63241001e-02  8.97206420e-03  3.87432721e-02 ... -5.18723923e-03
  -6.76066126e-04 -6.92022322e-02]
 [-2.07396298e-02  3.07961976e-02  8.08481747e-03 ... -2.39468991e-02
  -4.44283926e-05 -3.59766789e-02]
 ...
 [-3.70793644e-02 -1.52627076e-02  2.31076298e-03 ...  1.33535544e-02
   1.03392826e-02 -1.18807079e-02]
 [-2.70575635e-04  1.35694923e-02 -2.23894293e-02 ... -1.42009744e-03
   2.40530163e-02  8.37217236e-03]
 [ 3.16858520e-02 -1.00778044e-02  1.55174258e-02 ...  3.05504650e-03
   2.21377918e-02  8.48616967e-03]]
1 <class 'numpy.ndarray'> [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0

/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:150: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'x', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(


restrict_active_set finished
cur_objective finished
this->active_set.size()45this->active_set{0, 1}
{0, 2}
{0, 3}
{0, 4}
{0, 5}
{0, 6}
{0, 7}
{0, 8}
{0, 9}
{1, 2}
{1, 3}
{1, 4}
{1, 5}
{1, 6}
{1, 7}
{1, 8}
{1, 9}
{2, 3}
{2, 4}
{2, 5}
{2, 6}
{2, 7}
{2, 8}
{2, 9}
{3, 4}
{3, 5}
{3, 6}
{3, 7}
{3, 8}
{3, 9}
{4, 5}
{4, 6}
{4, 7}
{4, 8}
{4, 9}
{5, 6}
{5, 7}
{5, 8}
{5, 9}
{6, 7}
{6, 8}
{6, 9}
{7, 8}
{7, 9}
{8, 9}
 this->active_set_size.size() 9
 this->active_set_size.capacity() 100
 this->active_set_size.push_back SLEEP 1000